# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_app_history 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('music_app_history')
except Exception as e:
    print(e)

##  queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
### sessionId = 338, and itemInSession = 4
### Create a table to query based on sessionId and itemInSession. 
### Choose sessionId as Primary key and itemInSession as clustering key, which will keep all artist data pertaining to
### a session in same partition.
###  Define Primary key => sessionId first and then the clustering key => itemInSession next before defining other fields

In [9]:
query = "drop table if exists artist_title_song_length"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "create table  artist_title_song_length (sessionId bigint, itemInSession bigint,artist text, song text, length float, PRIMARY KEY (sessionId, ItemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

# insert data  into table artist_title_song_length

In [10]:

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into artist_title_song_length(sessionId, itemInSession, artist, song, length) values (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

###  SELECT to verify that the data have been inserted into each table
## sessionId will define the partition and itemInSession the order in which data is shown.

In [11]:

query = "select artist, song as song_title, length from artist_title_song_length where sessionId=338 and itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
 
for row in rows:
    print (row.artist, row.song_title, row.length)
               

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
## for userid = 10, sessionid = 182
### for Query 2 
### define Primary key first, Clustering keys next before any other fields
### Use userId and session Id as partition key to keep all of user's sessions in the same node. 
### This way query for a user id and session id will be faster. Use itemInSession as clustering key
### Primary Key determines the partition and clustering key determines the order of data in these partitions.
### As we want data to be ordered by only itemInSession, create composite key using userid and sessionid and #### make itemInSession clustering key 


In [12]:

query = "drop table if exists artist_song_user_details"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "create table  artist_song_user_details (userid bigint, sessionId bigint, itemInSession bigint,artist text, song text, firstname text, lastname text,PRIMARY KEY ((userId, sessionId), ItemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
               

In [13]:
file = 'event_datafile_new.csv'

# insert primary key first, clustering key1, clustering key2 before any other fields.
query = "insert into artist_song_user_details (userid, sessionId, itemInSession,artist, song, firstname, lastname) values (%s, %s, %s, %s, %s, %s, %s)"
        
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4])) 
                    

### Create query to select artist, song and user details using userid and session id ordered by itemInSession

In [14]:
query = "select artist, song, firstname, lastname from artist_song_user_details where userid=10 and sessionid=182 order by  itemInSession"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
 
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
### As data required is user who listened to a particular song, lets keep data for a particular song in 
### same partition.
### Define Primary key as song and user id as clustering key




In [15]:
query = "drop table if exists song_to_user_details"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
query = "create table  song_to_user_details (song text, userId bigint,firstname text, lastname text, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)
                    

                    

### Insert data in to song_user_details table. Insert song and user id first

In [16]:
file = 'event_datafile_new.csv'
query = "insert into song_to_user_details(song, userId, firstname, lastname) values (%s, %s, %s, %s)"

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
      session.execute(query, (line[9], int(line[10]), line[1], line[4])) 
                    

### Query to get user details who listened to a particular song

In [17]:
query = "select firstname, lastname from song_to_user_details where song='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
 
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the table before closing out the sessions

In [18]:
query = "drop table if exists artist_title_song_length"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table artist_song_user_details"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
        
query = "drop table song_to_user_details"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()